# Introduction

This notebook runs a model that computes the velocity profile of a sphere falling through a two-layer stratification. The entrainment by the upper layer is modeled as a sphere of radius $A^*$, which can be adjusted.

The advection of the interface is done with the Euler method which will solve the system of equations describing the fluid flow $\mathscr{v}(x) = \mathscr{u}(x)+\mathscr{w}(x)$. The stratification induced flow, $\mathscr{w}$, can be controlled.


This notebook plots the interface over time, velocity versus time and position, as well as other metrics.

# Importing Libraries and Save to Path function



In [ ]:
# # mount drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from os import makedirs,path
from errno import EEXIST

from datetime import datetime, timezone, date

In [ ]:
def mkdir_p(mypath):
    # Creates a directory: same as mkdir or fulltime(path)
    try:
        makedirs(mypath)
    except OSError as exc:
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise




# Creating directory for a path

#filename = 'Figures/VariableSweeps/h={h}/Ni={Ni}/N={N}/rhos={rhos}/rhot={rhot}/rhob={rhob}'.format(h=h, Ni=Ni, N=N, rhos=rhos, rhot=rhot, rhob=rhob) #path for fig save

now = datetime.now()


datestring = now.strftime('%Y_%m_%d')
timestring= now.strftime("%H_%M")




fulltime = datetime.today().strftime('%Y-%m/%d/%H_%M')


filename = '/content/gdrive/MyDrive/Output/{fulltime}'.format(fulltime=fulltime) #path for fig save


# filename = '/content/gdrive/MyDrive/Output/Anfsweep/{fulltime}'.format(fulltime=fulltime) #path for fig save


mkdir_p(filename)

In [ ]:
# filename = '/content/drive/MyDrive/Output'


In [ ]:
# now = datetime.now()


# datestring = now.strftime('%Y_%m_%d')
# timestring= now.strftime("%H_%M")




# fulltime = datetime.today().strftime('%Y-%m/%d/%H_%M')

# {fulltime}'.format(fulltime=fulltime)



# Variable definitions

We make a simple model testing the singularity analysis of W FF

In [ ]:
dt= 1.0   # time step


Tmax = 90

Ndt = 100000

#Ndt = int(Tmax/dt) #number of time steps
Ni = int(10)   #number of interfacial points close to sphere
Nj = int(Ni/2.0)          # number of interfacial points far from the sphere







#=============================================================================
# Physical Constants Exp. Data


# Microplastics

# R0 = 1.5
# A = 0.641
# rhos = 1.22

# rhot = 0.997
# rhob = 1.0

# mu_top = 0.09107
# mu_bottom = 0.15673

# mu = 0.5*(mu_top+mu_bottom)

R0 = 5.4

A = 0.641

rhos = 1.3

rhot= 1.27


rhob = 1.284


mu = 4.773326883

# =============================================================================
# Exp 8: 05-20-2016 Stokes
# Sphere name: 8 --- Salt = NaCL
# Sphere recovers, rhoregion = 0.21252153934520693
# =============================================================================


# A = 0.641
# rhos = 1.36712

# rhot = 1.34629
# rhob=1.35

# # mutop = 5.03263372
# # mubottom = 5.276099152


# mutop = 4.773326883
# mubottom = 5.000284943


# # mu = (mutop+mubottom)/2.0

# mu = mutop

# =============================================================================
# Exp 1: 04-09-2013 Entrainment
# Sphere name: 8 --- Salt = NaCL
#
# =============================================================================


# A = 0.641
# rhos = 1.36712

# rhot =1.34695
# rhob= 1.36178

# mutop = 3.611845768
# mubottom =3.406853812

# mu = (mutop+mubottom)*0.5

# mu = mutop

rhoref = (rhot+rhob)/2.0

g = 981.0


y_start = -1.0 # initial flow line Y=Z=x3 value
#=============================================================================


epsilon = 1e-9

ANF = 2.0*A


nzeta = 10
nrho = 10
ntheta = 20

rhoregion = 0 #(rhob-rhos)/(rhot-rhob)



frtd = 4.0/3.0
hf = 0.5
thd = 1/3.0
sxth = 1/6.0
twonths = 2.0/9.0
svnhf = 7.0/ 2.0


oneoversixmuA = 1.0/ (6.0*np.pi*mu*A)
govereightpimu = g / (8.0*np.pi*mu)

In [ ]:
mu

4.773326883

In [ ]:
V_TermTop =  (2*A**2 *g)/(9*mu)*(rhot-rhos)
V_TermBttm = (2*A**2 *g)/(9*mu)*(rhob-rhos)

print('V Term Top:', V_TermTop)
print('V Term Bottom :', V_TermBttm)

V Term Top: -0.5629536392259692
V Term Bottom : -0.3002419409205169


In [ ]:
# Reynolds Number

U =- V_TermTop
Re = rhos*A*U/ mu

print('Reynolds Number is approx', Re)

Reynolds Number is approx 0.09827721399883858


# Array Initialization

In [ ]:
X = np.zeros( (Ndt, Ni+Nj))
X_1 = np.power(np.linspace(0, np.sqrt(2.0*A), Ni), 2)
X_2 = np.linspace(np.sqrt(2.001**2 *A**2), R0, Nj)


X[0,:] = np.concatenate((X_1, X_2) )


# Initialization of Y - Array

Y = np.zeros( (Ndt, Ni+Nj))
Y[0,:] = y_start* np.ones_like(X[0,:])



# Initialization of radius array

r = np.zeros( (Ndt, Ni+Nj))



# Sphere Velocity Function Definitions


In [ ]:

#Volumes by region

def Vol_s(A):
    return frtd*np.pi*A**3 # Absolute sphere volume

def Vol_s_top_ent(A, yend):
    return 2*np.pi*(thd*A**3 - hf*(A**2)*yend+sxth*(yend**3)) #volume of sphere in top in entrainment

def Vol_s_bttm_ent(A, yend):
    return 2*np.pi*(hf*(A**2)*yend-sxth*(yend**3)+thd*(A**3)) #volume of sphere in bottom in entrainment

# Gravity Force

def GravityForce(yend, A):
    return rhos*Vol_s(A)*g

# Buoyancy Force within each region of Omega_f

def B_bttm_ent(A, yend):
    return thd*g*np.pi*rhob*(2*A-yend)*(A+yend)**2 # when yend < A in top layer

def B_top_ent(A, yend):
    return thd*g*rhot*np.pi*(2*A+yend)*(A-yend)**2 ## when -A < yend in bottom layer


def B_int(A, yend):

    return float(rhot*g*Vol_s_top_ent(A, yend)+g*rhob*Vol_s_bttm_ent(A, yend))


    #return float(B_top_ent(A, yend)+B_bttm_ent(A, yend)) #when -A < yend < A

# Defining Buoancy Function

def Buoyancy(yend, A):

    if abs(yend)<A:
        return B_int(A, yend)
    if yend < -A:
        return g*rhot*Vol_s(A)
    else:
        return g*rhob*Vol_s(A)

# This is the old S_int

def S_int(yend, A):


    if yend < -A:

        return g*(rhot-rhob)*(-1.0/(6.0*Astar**3))*( A*(A**2-3*Astar**2)*np.pi*(-y0**3+3*Astar**2*(y0-yend)+yend**3))

    elif abs(yend)<= A:

        return g*(rhot-rhob)*(np.pi*(9*A*Astar**4*y0-3*A*Astar**2*y0**3+A**3*(4*Astar**3-3*Astar**2*y0+y0**3)+3*A*(A-Astar)*Astar**2*(A+3*Astar)*yend-(A-Astar)**2 *(A+2*Astar)*yend**3))/ (6.0*Astar**3)

    elif A <= yend <= Astar:
        return g*(rhot-rhob)*(A *np.pi *(3.0*Astar**2 *(-y0**3 + 3.0 *Astar**2 *(y0 - yend) + yend**3) + A**2 *(8.0 *Astar**3 + y0**3 - yend**3 + 3.0 *Astar**2 *(-y0 + yend))))/(6.0* Astar**3)

    else:

        return g*(rhot-rhob)*(


        (A*np.pi/6.0)*(

        -(3.0*(Astar-y0)**2 *(2*Astar+y0))/(Astar) + A**2 *(   10 - (3.0*Astar)/ (np.sqrt(A**2-2*A*Astar+2*Astar**2)) -(3.0*y0)/Astar  + (y0**3 / Astar**3) + (3.0*yend)/(np.sqrt( (A-Astar)**2 + yend**2)))

            + 9*(A-Astar)**2 *(np.arcsinh(Astar / abs(A-Astar)) - np.arcsinh(yend/ abs(A-Astar)))))


V_TermTop =  (2*A**2 *g)/(9*mu)*(rhot-rhos)
V_TermBttm = (2*A**2 *g)/(9*mu)*(rhob-rhos)




def Velocity(yend):

    if yend <= -A:

        return V_TermTop + oneoversixmuA * S_int(yend, A) #terminal velocity of sphere in top

    elif abs(yend) <= A:
        return oneoversixmuA*(S_int(yend, A) + Buoyancy(yend,A)-GravityForce(yend,A)) #velocity in entrainment layer

    elif yend >= A:

        return V_TermBttm  + oneoversixmuA*S_int(yend, A)  #terminal velocity of sphere in bottom













In [ ]:
oneoversixmuA

0.017338825308148103

In [ ]:
GravityForce(1, A)*oneoversixmuA

24.39465769979198

In [ ]:
V_TermTop

-0.5629536392259692

In [ ]:
V_TermBttm

-0.3002419409205169

# Stokes Velocity Function Definitions

In [ ]:
def ux(yend, X, Y):

    #print('X_x=', X, 'and ', 'Y_x=', Y)

    V= -Velocity(yend)

    rr = np.sqrt(X**2+Y**2)

    ux = V*(((-3*A*X*Y)/(4*rr**3))+ ((3*(A**3)*X*Y))/(4*rr**5))   #R or x-comp of stokes flow

    return ux



def uy(yend, X, Y):

    V = -Velocity(yend)

    rr = np.sqrt(X**2+Y**2)

    uy =  V*(1+(-3*A/(4*rr)-3*A*Y**2/(4*rr**3)-A**3/(4*rr**3)+3*Y**2*A**3/(4*rr**5)))  #Z or y-comp of stokes flow

    return uy

# Full Field Function Definitions



In [ ]:
def Oseen_def(X,Y, rho, zeta, theta):

  #print('X=', X)


  def rr(X, Y, rho, zeta, theta):

      rr = np.sqrt(X**2 + rho**2 - 2*rho*X*np.cos(theta)+(Y-zeta)**2)

      return rr

  def rstar(X, Y, rho, zeta, theta):

      rstar = np.sqrt(sigma(rho,zeta)**2 *(zeta**2+rho**2) + X**2 - 2*sigma(rho,zeta)*(X*rho*np.cos(theta)-Y*zeta) + Y**2)

      return rstar

  def sigma(rho, zeta):

      if rho**2 + zeta**2 <=0:
          sigma = 0
      else:
          sigma =  (A**2 / (rho**2+zeta**2))

      return sigma

  def modx(X, Y):

      modx = np.sqrt(X**2 + Y**2)

      return modx

  def modystar(rho,zeta):

      if rho**2+zeta**2 == 0:

          modystar = 0

      else:

          modystar = A**2 / np.sqrt(rho**2+zeta**2)

      return modystar

  def dPhi3dx1(rho, zeta, theta, X, Y):

    dPhi3dx1 = ((A**2 - X**2 - Y**2)*(-A**2 + zeta**2 + rho**2*np.cos(theta)**2)* ((-3*A*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))* (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**2.5 - (3*zeta*(X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/(A*((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5) + (2*zeta*((3*A**2*zeta*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))* (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/ ((zeta**2 + rho**2*np.cos(theta)**2)* ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**2.5) +  (3*A**2*rho*np.cos(theta)*(X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)/ ((zeta**2 + rho**2*np.cos(theta)**2)* ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**2.5) - (A**2*rho*np.cos(theta))/  ((zeta**2 + rho**2*np.cos(theta)**2)* ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5)))/A+ (3*A*((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))* (((np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))* (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/ ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5 + (A**2*zeta*(X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/ ((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)) - ((Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))* ((A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2 -  (((A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + (X*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))/np.sqrt(X**2 + Y**2))*(Y - (2*A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) - (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) + (A**2*zeta*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))/((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2 + (2*(Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/ ((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))* ((A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + (X*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))/np.sqrt(X**2 + Y**2))*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**3 - (A**2*X*zeta*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/(np.sqrt(X**2 + Y**2)*(zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2)))/(np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))) - (3*A*((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))*((A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))* ((Y - (2*A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) - (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/ np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) +  (A**2*zeta*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))/ ((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)) - ((Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2))/(np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))**2) + (3*A*((A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + (X*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))/np.sqrt(X**2 + Y**2))*((Y - (2*A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) - (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) + (A**2*zeta*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))/((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)) - ((Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2))/(np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))))/(2.0*(zeta**2 + rho**2*np.cos(theta)**2)**1.5)


    return dPhi3dx1


  # d phi / d x3

  def dPhi3dx3(rho,zeta, theta, X, Y):

      dPhi3dx3 =  ((A**2 - X**2 - Y**2)*(-A**2 + zeta**2 + rho**2*np.cos(theta)**2)* ((-3*A*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2)/ ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**2.5 +  A/((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5 - (3*zeta*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/(A*((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5) + (2*zeta*((3*A**2*zeta*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2)/ ((zeta**2 + rho**2*np.cos(theta)**2)* ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**2.5) +  (3*A**2*rho*np.cos(theta)*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))*(X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2)))/ ((zeta**2 + rho**2*np.cos(theta)**2)* ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**2.5) - (A**2*zeta)/((zeta**2 + rho**2*np.cos(theta)**2)* ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5)))/A + (3*A*((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))* ((1 + (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2)/ ((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5 -  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) + (A**2*zeta*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/ ((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)) - ((Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/ ((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))* ((A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/ np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2 -  (((A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (Y*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))/ np.sqrt(X**2 + Y**2))* (Y - (2*A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) - (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*(Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) + (A**2*zeta*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))/ ((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2 + (2*(Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/ ((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))* ((A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (Y*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))/ np.sqrt(X**2 + Y**2))* (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -   (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**3 -  ((1 + (A**2*Y*zeta)/ (np.sqrt(X**2 + Y**2)*(zeta**2 + rho**2*np.cos(theta)**2)*  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))* (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +   (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +   (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2))/ (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +   (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))) - (3*A*((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))* ((A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))* ((Y - (2*A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) -  (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/  np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) +  (A**2*zeta*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))/((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))/((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)) -  ((Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/ ((zeta**2 + rho**2*np.cos(theta)**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))* (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2))/ (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*  (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))**2) +  (3*A*((A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (Y*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))/np.sqrt(X**2 + Y**2))*((Y - (2*A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) -  (np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2)))/ np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2) +  (A**2*zeta*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 +  (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2))/ ((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt(X**2 + Y**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)) - ((Y + (A**2*np.sqrt(X**2 + Y**2)*zeta)/((zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)))*(-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) -  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) +  (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))/ ((A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) +  np.sqrt(X**2 + Y**2)* np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2))**2))/(np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)* (-((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2) - (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 +  (A**2*Y*zeta)/(zeta**2 + rho**2*np.cos(theta)**2) + (A**2*X*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2) + np.sqrt((A**4*zeta**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2 + (A**4*rho**2*np.cos(theta)**2)/(zeta**2 + rho**2*np.cos(theta)**2)**2)*np.sqrt((Y - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)))))/(2.0*(zeta**2 + rho**2*np.cos(theta)**2)**1.5)

      return dPhi3dx3

  def Oseen_x(X, Y, rho, zeta, theta):

    #singularity_boolean = 1.0*((((X-rho)**2 + (Y-zeta)**2) > epsilon))


    singularity_boolean = 1.0*(((((X-rho)**2 + (Y-zeta)**2) > epsilon) ))

    #singularity_boolean = 1.0*(((((X-rho)**2+(Y-zeta)**2)) > epsilon) & (X>epsilon))



    sing_indicies = np.where(singularity_boolean)[0]

    rr = np.sqrt(X[sing_indicies]**2 + rho**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2 - 2*X[sing_indicies]*rho*np.cos(theta))

    rs = (A**2 / (rho**2+zeta**2))*rr

    #modx = np.sqrt(X[sing_indicies]**2 + Y[sing_indicies]**2)
    mody = np.sqrt(rho**2+zeta**2)





    Full_Wx = np.zeros(len(X))

    #Full_Wx[sing_indicies] =  ((Y[sing_indicies] - zeta)*(X[sing_indicies] - rho*np.cos(theta)))/(rr**3) # Stokeslet

    # Full_Wx[sing_indicies] =  ((Y[sing_indicies] - zeta)*(X[sing_indicies] - rho*np.cos(theta)))/(r**3) - (A**3 * (X[sing_indicies]-(A**2 / (rho**2+zeta**2))*rho*np.cos(theta))*(Y[sing_indicies]-(A**2 / (rho**2+zeta**2))*zeta))/rs**3

    Full_Wx[sing_indicies] =  ((Y[sing_indicies] - zeta)*(X[sing_indicies] - rho*np.cos(theta)))/(rr**3) #-  (A**3 / mody**3) * (X[sing_indicies]-(A**2/mody**2)*rho*np.cos(theta))*(Y[sing_indicies]-(A**2/mody**2)*zeta)/rs**3




    return Full_Wx

  # Full Oseen in Z, y, x3 direction

  def Oseen_y(X, Y, rho, zeta, theta):

    singularity_boolean = 1.0*(((((X-rho)**2 + (Y-zeta)**2) > epsilon) ))



    #singularity_boolean = 1.0*(((((X-rho)**2+(Y-zeta)**2)) > epsilon) & (X>epsilon))

    sing_indicies = np.where(singularity_boolean)[0]

    #print(sing_indicies)

    Full_Wy = np.zeros(len(X))

    rr = np.sqrt(X[sing_indicies]**2 + rho**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2 - 2*X[sing_indicies]*rho*np.cos(theta))

    modx = np.sqrt(X[sing_indicies]**2 + Y[sing_indicies]**2)
    mody = np.sqrt(rho**2+zeta**2)

    rs = (A**2 / (rho**2+zeta**2))*rr


    #Full_Wy[sing_indicies] =   (Y[sing_indicies] - zeta)**2/(rr**3)+ 1/rr

    Full_Wy[sing_indicies] = (Y[sing_indicies] - zeta)**2/(rr**3)+ 1/rr #- A/(mody*rs) -(A**3/mody**3)*(Y[sing_indicies]-(A**2 / (rho**2+zeta**2))*zeta)**2 /(rs**3)


    #Full_Wy[sing_indicies] =   (Y[sing_indicies] - zeta)**2/(rr**3)+ 1/rr - (A**3*(Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2)/((zeta**2 + rho**2*np.cos(theta)**2)**1.5*((Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X[sing_indicies] - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5) - A/(np.sqrt(zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X[sing_indicies] - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)) + (A*zeta*(A**2 - zeta**2 - rho**2*np.cos(theta)**2)*(zeta*(A**4 - 2*A**2*Y[sing_indicies]*zeta + (X[sing_indicies]**2 + Y[sing_indicies]**2)*zeta**2) - rho**2*(2*A**2*Y[sing_indicies] - (X[sing_indicies]**2 + Y[sing_indicies]**2)*zeta)*np.cos(theta)**2))/((zeta**2 + rho**2*np.cos(theta)**2)**2.5*(A**4 - 2*A**2*Y[sing_indicies]*zeta + (X[sing_indicies]**2 + Y[sing_indicies]**2)*zeta**2 - 2*A**2*X[sing_indicies]*rho*np.cos(theta) + rho**2*(X[sing_indicies]**2 + Y[sing_indicies]**2)*np.cos(theta)**2)*np.sqrt((Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X[sing_indicies] - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)) +dPhi3dx3(X[sing_indicies], Y[sing_indicies], rho, zeta, theta) - (A**3*(Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2)/((zeta**2 + rho**2*np.cos(theta)**2)**1.5*((Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X[sing_indicies] - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)**1.5) - A/(np.sqrt(zeta**2 + rho**2*np.cos(theta)**2)*np.sqrt((Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X[sing_indicies] - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)) + (A*zeta*(A**2 - zeta**2 - rho**2*np.cos(theta)**2)*(zeta*(A**4 - 2*A**2*Y[sing_indicies]*zeta + (X[sing_indicies]**2 + Y[sing_indicies]**2)*zeta**2) - rho**2*(2*A**2*Y[sing_indicies] - (X[sing_indicies]**2 + Y[sing_indicies]**2)*zeta)*np.cos(theta)**2))/((zeta**2 + rho**2*np.cos(theta)**2)**2.5*(A**4 - 2*A**2*Y[sing_indicies]*zeta + (X[sing_indicies]**2 + Y[sing_indicies]**2)*zeta**2 - 2*A**2*X[sing_indicies]*rho*np.cos(theta) + rho**2*(X[sing_indicies]**2 + Y[sing_indicies]**2)*np.cos(theta)**2)*np.sqrt((Y[sing_indicies] - (A**2*zeta)/(zeta**2 + rho**2*np.cos(theta)**2))**2 + (X[sing_indicies] - (A**2*rho*np.cos(theta))/(zeta**2 + rho**2*np.cos(theta)**2))**2)) +dPhi3dx3(X[sing_indicies], Y[sing_indicies], rho, zeta, theta)

    #-(mod(X[sing_indicies], Y[sing_indicies])-A**2)*dPhidx3(rho,zeta,theta,X[sing_indicies],Y[sing_indicies])+
    return Full_Wy


  #print(Oseen_x(rho, zeta, theta))

  #print('Wx=', Oseen_x(rho,zeta,theta))


  return Oseen_x(X, Y, rho, zeta, theta), Oseen_y(X,Y, rho, zeta, theta)

# Full Field Integration


In [ ]:
def trapz(W,a,b,n, XT):


    h = (b-a)/(n-1)
    x = a
    r = np.zeros(len(XT)) # initial value of y = int_a^b
    for j in range(1,n-1):
        x +=h
        r = r + W(x)
    r = (r + (W(a)+W(b))/2)*h

    return r #returns a row vector of length XT



# def sph(zeta):

#     #print('A and zeta is:', A, zeta)
#     if A**2 - zeta**2 < 0:

#         #print('A**2-zeta**2 sphere zeta returns 0')

#         return 0.0

#     else:

#         sphere=  np.sqrt(A**2-zeta**2)

#         #print('sphere(zeta) returns', sphere)

#         return sphere

# def shell(Astar, zeta):

#     if zeta <= - Astar :

#        # print('shell(zeta) when zeta<-Astar=', 0)

#         return 0

#     elif abs(zeta) < Astar:

#         #print('Astar=', Astar, 'zeta=', zeta)

#         shellzeta = np.sqrt(Astar**2-zeta**2)

#         #print('shellzeta when |zeta|<Astar=', shellzeta)

#         return shellzeta

#         #return 0


#     else: #stem
#         #print('Astar=', Astar)
#         stem = abs(Astar-A)
#         #print('stem', stem)

#         return 0
#         #return stem


# def sph_shell(A, zeta):
#     if A**2 - zeta**2 < 0:
#         #print('A**2-zeta**2<0')
#         return 0
#     else:
#         return np.sqrt(A**2-zeta**2)


def sph(zeta):
    if A**2 - zeta**2 <= 0:
        return 0

    else:
        return np.sqrt(A**2-zeta**2)


def shell(Astar, zeta):
    # print("-------------")
    # print('shell: zeta', zeta, '\n', 'Astar', Astar)
    # print("-------------")

    if zeta <= - Astar:
        return 0

    elif abs(zeta) < Astar:

        return np.sqrt(Astar**2-zeta**2)
        #return 0
    else:
        return abs(Astar-A)

    # change bounds in integration

def sph_shell(A, zeta):
    if A**2 - zeta**2 < 0:
        return epsilon
    else:
        return np.sqrt(A**2-zeta**2)

In [ ]:
def wy_full(X, Y):


    def Wtheta(theta):
        def Wz(zeta):
            def Wrho(rho):
                #from EulerMethod_n import X, Y, i, y0
                #singularity_boolean = 1.0*(((X**2+rho**2-2*rho*X*np.cos(theta) +(Y-zeta)**2)) > epsilon)

                Oseen_x, Oseen_y = Oseen_def(X, Y, rho, zeta, theta)


                Wr =rho*Oseen_y


                #print('Wr[-1]=', Wr[-1], 'sph_shell at Astar', sph_shell(Astar, zeta),"zeta",zeta)
                #print("rem",removable_sing,"Wr", Wr)
                return Wr
            #print("Astar=", Astar, 'zeta=',zeta)

            #print('shell(A, zeta)=', shell(Astar, zeta), 'shell(Astar, zeta)=', shell(Astar,zeta))
            #print('wy: zeta=', zeta)

            #print('sph(zeta)=', sph(zeta), 'shell(Astar, zeta)=', shell(Astar,zeta))

            #Wzeta = trapz(Wrho, shell(A, zeta), shell(Astar, zeta) , nrho, X) # correct
            Wzeta = trapz(Wrho, sph(zeta), shell(Astar, zeta) , nrho, X) # correct

            #print('Wzeta=', Wzeta)
            return Wzeta


        Wtheta=trapz(Wz, y0, yend, nzeta, X)

        return Wtheta

    w = trapz(Wtheta, 0, 2*np.pi, ntheta, X)

    #print('wy_full', w)

    return -(rhot-rhob)*govereightpimu*w




# Integration of full Oseen in X=R=x1 direction

def wx_full(X, Y):


    def Wtheta(theta):
        def Wz(zeta):
            def Wrho(rho):

                #singularity_boolean = 1.0*((X**2+rho**2-2*rho*X*np.cos(theta) +(Y-zeta)**2 )>= epsilon)

                Oseen_x, Oseen_y = Oseen_def(X, Y, rho, zeta, theta)



                Wr =rho*Oseen_x

                #print("rem",removable_sing,"Wr", Wr)
                return Wr
            #print('wx: zeta=', zeta)
            #Wzeta = trapz(Wrho, shell(A, zeta), shell(Astar, zeta), nrho, X) #correct

            #print('sph(zeta)=', sph(zeta), 'shell(Astar, zeta)=', shell(Astar,zeta))

            Wzeta = trapz(Wrho, sph(zeta), shell(Astar, zeta), nrho, X) #correct


            return Wzeta
        Wtheta=trapz(Wz, y0, yend, nzeta, X)
        return Wtheta

    w = trapz(Wtheta, 0, 2*np.pi, ntheta, X)

    #print('wx_full=', w)


    return -(rhot-rhob)*govereightpimu*w

# Far Field Function Definitions




In [ ]:
def lamb(rho, zeta, X, Y):
    #from EulerMethod_n import r, Astar, y0, yend

    #print('lamb numerator:', 4*X*rho)
    #print('lamb denominator:', (X**2 -2*rho*X + Y**2 + zeta**2 + rho**2 - 2*Y*zeta))

    lambd =  (-4*X*rho)/ (X**2 -2*rho*X + Y**2 + zeta**2 + rho**2 - 2*Y*zeta)
    #print('lambd:', lambd)
    return lambd

def K(argument):

    #return 1.0

    return sp.special.ellipk(argument) # complete integral of second kind

def E(argument):

    #return 1.0

    return sp.special.ellipe(argument) # incomplete integral of second kind

In [ ]:
def W_FFx(rho, zeta, X, Y):


    singularity_boolean = 1.0*(((((X-rho)**2+(Y-zeta)**2)) > epsilon) & (X>epsilon))

    #singularity_boolean = 1.0*((((X**2 -2*rho*X + Y**2 + zeta**2 + rho**2 - 2*Y*zeta)) > epsilon) & (X>epsilon))

    #singularity_boolean = 1.0*(((((X-rho)**2+(Y-zeta)**2)) > epsilon) )


    #singularity_boolean = 1.0*((( X**2 + (Y - zeta)**2 + rho**2 - 2 *X*rho)>epsilon) & (X>epsilon))

    sing_indicies = np.where(singularity_boolean)[0]

    #print('Sing Indicies:', sing_indicies)
    #print('singularity in WFFx is:', ((X-rho)**2+(Y-zeta)**2))



    W_FFx = np.zeros(len(X))





    #print('wffx1 =', W_FFx)
    #W_FFx[0] = ux(yend)
    #X = X[sing_indicies]
    #Y = Y[sing_indicies]

    #print('W_FFx shape is :', W_FFx.shape)

    argument=lamb(rho, zeta, X[sing_indicies], Y[sing_indicies])

    #argument = 0
    #print()

    #W_FFx[sing_indicies] = (3*A*X[sing_indicies]*Y[sing_indicies]*(2*zeta**2 + rho**2))/(16.0*(X[sing_indicies]**2 + Y[sing_indicies]**2)**1.5*mu*(zeta**2 + rho**2)**1.5) + (3*A**5*X[sing_indicies]*(8*zeta**3*(2*X[sing_indicies]**4 - 2*Y[sing_indicies]**2*(Y[sing_indicies] - 4*zeta)*(2*Y[sing_indicies] - zeta) + X[sing_indicies]**2*(-2*Y[sing_indicies]**2 - 17*Y[sing_indicies]*zeta + 2*zeta**2)) + 8*zeta*(-3*X[sing_indicies]**4 + Y[sing_indicies]**2*(6*Y[sing_indicies]**2 - 33*Y[sing_indicies]*zeta - 4*zeta**2) + X[sing_indicies]**2*(3*Y[sing_indicies]**2 + 37*Y[sing_indicies]*zeta + zeta**2))*rho**2 + (4*Y[sing_indicies]**2*(3*Y[sing_indicies] + 8*zeta) - X[sing_indicies]**2*(23*Y[sing_indicies] + 8*zeta))*rho**4))/(128.0*(X[sing_indicies]**2 + Y[sing_indicies]**2)**3.5*mu*(zeta**2 + rho**2)**3.5) + (A**3*X[sing_indicies]*(X[sing_indicies]**2*(Y[sing_indicies] + 5*zeta)*(2*zeta**2 - rho**2) + Y[sing_indicies]*(Y[sing_indicies]**2*(2*zeta**2 - rho**2) + 10*Y[sing_indicies]*zeta*(-2*zeta**2 + rho**2) + 3*(2*zeta**4 + 3*zeta**2*rho**2 + rho**4))))/(16.0*(X[sing_indicies]**2 + Y[sing_indicies]**2)**2.5*mu*(zeta**2 + rho**2)**2.5) - ((Y[sing_indicies] - zeta)*(2*(X[sing_indicies]**2 - Y[sing_indicies]**2 + 2*Y[sing_indicies]*zeta - zeta**2 - rho**2)*E(argument) + 2*(X[sing_indicies]**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2 + 2*X[sing_indicies]*rho + rho**2)*K(argument)))/(X[sing_indicies]*np.sqrt(X[sing_indicies]**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2 - 2*X[sing_indicies]*rho + rho**2)*(X[sing_indicies]**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2 + 2*X[sing_indicies]*rho + rho**2))


    W_FFx[sing_indicies] = 2.0 * (Y[sing_indicies] - zeta) / (np.pi * X[sing_indicies] * np.sqrt((X[sing_indicies] - rho) ** 2 + (Y[sing_indicies] - zeta) ** 2) *((X[sing_indicies] + rho) ** 2 + (Y[sing_indicies] - zeta) ** 2)) * ((X[sing_indicies] ** 2 - rho ** 2 - (Y[sing_indicies] - zeta) ** 2) * E(argument) + ((rho + X[sing_indicies]) ** 2 + (Y[sing_indicies] - zeta) ** 2) * K(argument)) - 3.0 * A * X[sing_indicies] * Y[sing_indicies] * (2.0 * zeta ** 2 + rho ** 2) / (2.0 * np.sqrt(X[sing_indicies] ** 2 + Y[sing_indicies] ** 2) ** 3 * np.sqrt(zeta ** 2 + rho ** 2) ** 3) + (A ** 3 * X[sing_indicies] * (X[sing_indicies] ** 2 * (Y[sing_indicies] + 5.0 * zeta) * (2.0 * zeta ** 2 - rho ** 2) +Y[sing_indicies] * (Y[sing_indicies] ** 2 * (2.0 * zeta ** 2 - rho ** 2) + 10.0 * Y[sing_indicies] * zeta * (-2.0 * zeta ** 2 + rho ** 2) +3.0 * (2.0 * zeta ** 4 + 3.0 * zeta ** 2 * rho ** 2 + rho ** 4)))) / (2.0 * (X[sing_indicies] ** 2 + Y[sing_indicies] ** 2) ** (2.5) * (zeta ** 2 + rho ** 2) ** (2.5)) - (3.0 * A ** 5 * X[sing_indicies] * (8.0 * X[sing_indicies] ** 4 * (2.0 * zeta ** 3 - 3.0 * zeta * rho ** 2) + X[sing_indicies] ** 2 * (-8.0 * Y[sing_indicies] ** 2 * (2.0 * zeta ** 3 - 3.0 * zeta * rho ** 2) + Y[sing_indicies] * (-136.0 * zeta ** 4 + 296.0 * zeta ** 2 * rho ** 2 - 23.0 * rho ** 4) +8.0 * zeta * (2.0 * zeta ** 4 + zeta ** 2 * rho ** 2 - rho ** 4)) - 4.0 * Y[sing_indicies] ** 2 * (4.0 * Y[sing_indicies] ** 2 * (2.0 * zeta ** 3 - 3.0 * zeta * rho ** 2) + 8.0 * zeta * (2.0 * zeta ** 4 + zeta ** 2 * rho ** 2 - rho ** 4) -3.0 * Y[sing_indicies] * (12.0 * zeta ** 4 - 22.0 * zeta ** 2 * rho ** 2 + rho ** 4)))) / (16.0 * (X[sing_indicies] ** 2 + Y[sing_indicies] ** 2) ** (3.5) * (zeta ** 2 + rho ** 2) ** (3.5))



    #W_FFx[sing_indicies] = X[sing_indicies]
    #print('W_FFx shape is :', W_FFx.shape)
    # print('--------')
    #print('W_FFx2 =', W_FFx)
    # print('--------')

    return W_FFx

In [ ]:
def W_FFy(rho, zeta, X, Y):



    singularity_boolean = 1.0*(((((X-rho)**2+(Y-zeta)**2)) > epsilon) )

    #singularity_boolean = 1.0*((((X**2 -2*rho*X + Y**2 + zeta**2 + rho**2 - 2*Y*zeta)) > epsilon) & (X>epsilon))



    #singularity_boolean = 1.0*((( X**2 + (Y - zeta)**2 + rho**2 - 2 *X*rho)>epsilon) & (X>epsilon))


    sing_indicies = np.where(singularity_boolean)[0]

    #print('Singularity:', (((X-rho)**2+(Y-zeta)**2)))

    #print('X at sing indicies:', X[sing_indicies],',', 'Y at sing indicies:', Y[sing_indicies])


    W_FFy = np.zeros(len(X))



    #W_FFy.shape()
    #W_FFy[0]= uy(yend)
    #print('W_FFy initialized is :', W_FFy)
    #X = X[sing_indicies]
    #Y = Y[sing_indicies]

    #print('W_FFx shape is :', W_FFx.shape)
    #print('rho=', rho,':', 'zeta=', zeta)
    argument=lamb(rho, zeta, X[sing_indicies], Y[sing_indicies])

    #argument = 0
    #print()

    #print('argument:', argument)

    #W_FFy[sing_indicies] = (3*A*np.pi*X[sing_indicies]*Y[sing_indicies]*(rho**2 + 2*zeta**2))/(2.0*(X[sing_indicies]**2 + Y[sing_indicies]**2)**1.5*(rho**2 + zeta**2)**1.5) - (A**3*np.pi*X[sing_indicies]*(X[sing_indicies]**2*(Y[sing_indicies] + 5*zeta)*(-rho**2 + 2*zeta**2) + Y[sing_indicies]*(10*Y[sing_indicies]*zeta*(rho**2 - 2*zeta**2) + Y[sing_indicies]**2*(-rho**2 + 2*zeta**2) + 3*(rho**4 + 3*rho**2*zeta**2 + 2*zeta**4))))/(2.0*(X[sing_indicies]**2 + Y[sing_indicies]**2)**2.5*(rho**2 + zeta**2)**2.5) + (3*A**5*np.pi*X[sing_indicies]*(8*X[sing_indicies]**4*(-3*rho**2*zeta + 2*zeta**3) + X[sing_indicies]**2*(-8*Y[sing_indicies]**2*(-3*rho**2*zeta + 2*zeta**3) + Y[sing_indicies]*(-23*rho**4 + 296*rho**2*zeta**2 - 136*zeta**4) + 8*zeta*(-rho**4 + rho**2*zeta**2 + 2*zeta**4)) - 4*Y[sing_indicies]**2*(4*Y[sing_indicies]**2*(-3*rho**2*zeta + 2*zeta**3) + 8*zeta*(-rho**4 + rho**2*zeta**2 + 2*zeta**4) - 3*Y[sing_indicies]*(rho**4 - 22*rho**2*zeta**2 + 12*zeta**4))))/(16.0*(X[sing_indicies]**2 + Y[sing_indicies]**2)**3.5*(rho**2 + zeta**2)**3.5) - ((Y[sing_indicies] - zeta)*(2*(X[sing_indicies]**2 - rho**2 - Y[sing_indicies]**2 + 2*Y[sing_indicies]*zeta - zeta**2)*E(argument)+ 2*(X[sing_indicies]**2 + 2*X[sing_indicies]*rho + rho**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2)*K(argument)))/(X[sing_indicies]*np.sqrt(X[sing_indicies]**2 - 2*X[sing_indicies]*rho + rho**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2)*(X[sing_indicies]**2 + 2*X[sing_indicies]*rho + rho**2 + Y[sing_indicies]**2 - 2*Y[sing_indicies]*zeta + zeta**2))




    W_FFy[sing_indicies] = 4.0 * ((Y[sing_indicies] - zeta) ** 2 * E(argument) + ((X[sing_indicies] + rho) ** 2 + (Y[sing_indicies] - zeta) ** 2) * K(argument))/(np.pi * np.sqrt((X[sing_indicies] - rho) ** 2 + (Y[sing_indicies] - zeta) ** 2) * ((X[sing_indicies] + rho) ** 2 + (Y[sing_indicies] - zeta) ** 2)) - 3.0 * A * (X[sing_indicies] ** 2 + 2.0 * Y[sing_indicies] ** 2) * (2.0 * zeta ** 2 + rho ** 2)/(2.0 * np.sqrt(X[sing_indicies] ** 2 + Y[sing_indicies] ** 2) ** 3 * np.sqrt(zeta ** 2 + rho ** 2) ** 3) - A ** 3 / (2.0 * np.sqrt(X[sing_indicies] ** 2 + Y[sing_indicies] ** 2) ** 5 * np.sqrt(rho ** 2 + zeta ** 2) ** 5) * (X[sing_indicies] ** 4 * (-2.0 * zeta ** 2 + rho ** 2) - 2.0 * Y[sing_indicies] ** 2 * (2.0 * zeta ** 4 + 3.0 * zeta ** 2 * rho ** 2 +rho ** 4 + Y[sing_indicies] ** 2 * (2.0 * zeta ** 2 - rho ** 2) + 5.0 * Y[sing_indicies] * zeta * (-2.0 * zeta ** 2 + rho ** 2)) +X[sing_indicies] ** 2 * (2.0 * zeta ** 4 + 3.0 * zeta ** 2 * rho ** 2 + rho ** 4 + 5.0 * Y[sing_indicies] * zeta * ( -2.0 * zeta ** 2 + rho ** 2) + Y[sing_indicies] ** 2 * (-6.0 * zeta ** 2 + 3.0 * rho ** 2))) -  (3.0 * A ** 5 * (X[sing_indicies] ** 4 * (8.0 * zeta ** 4 - 24.0 * zeta ** 2 * rho ** 2 + 3.0 * rho ** 4 + 8.0 * Y[sing_indicies] * (  2.0 * zeta ** 3 - 3.0 * zeta * rho ** 2)) - 8.0 * Y[sing_indicies] ** 3 * ( 4.0 * zeta ** 5 + 2.0 * zeta ** 3 * rho ** 2 - 2.0 * zeta * rho ** 4 + Y[sing_indicies] ** 2 * (4.0 * zeta ** 3 - 6.0 * zeta * rho ** 2) - Y[sing_indicies] * ( 12.0 * zeta ** 4 - 22.0 * zeta ** 2 * rho ** 2 +rho ** 4)) - 8.0 * X[sing_indicies] ** 2 * Y[sing_indicies] * ( -6.0 * zeta ** 5 - 3.0 * zeta ** 3 * rho ** 2 + 3.0 * zeta * rho ** 4 +Y[sing_indicies] ** 2 * (2.0 * zeta ** 3 - 3.0 * zeta * rho ** 2) + Y[sing_indicies] * (22.0 * zeta ** 4 - 45.0 * zeta ** 2 * rho ** 2 +3.0 * rho ** 4)))) / (16.0 * (X[sing_indicies] ** 2 + Y[sing_indicies] ** 2) ** (3.5) * (zeta ** 2 + rho ** 2) ** (3.5))





    return W_FFy



# Far Field Integration


In [ ]:
def wx_FF(R, Z):

    def Wz(zeta):
        def Wrho(rho):

            Wr = rho*W_FFx(rho, zeta, R, Z)


            return Wr
            #return 0
        Wzeta = trapz(Wrho, sph(zeta), shell(Astar, zeta), nrho, Z)
        return Wzeta


    w = trapz(Wz, y0, yend, nzeta, R)

    #print('wx_ff=', w)

    return (rhot-rhob)*govereightpimu*w

In [ ]:
def wy_FF(R, Z):

  #print('initial shape of input R in wy_FF:', R.shape)
  #print('initial shape of input Z in wy_FF:', Z.shape)


  def Wz(zeta):

    def Wrho(rho):

      #print('The shape of Wr at beginning of wy_FF', Wr.shape)


      #Wr = np.zeros(len(R))

      #print('The shape of Wr before calling Wr=W_FFy is:', Wr.shape)
      #print('Wr before being assigned to W_FFy is: ', Wr)

      Wr = rho* W_FFy(rho, zeta, R, Z)

      #print('The shape of Wr after calling Wr=W_FFy is:', Wr.shape)
      #print("rem",removable_sing,"Wr", Wr)

      #print('shape of Wr in wy_FF:', Wr.shape)


      #fprint('Wrffy after assingment is', Wr)

      return Wr

  #return 0
    Wzeta = trapz(Wrho, sph(zeta), shell(Astar, zeta), nrho, R) #these xff will change name

  #print('Wzeta =', Wzeta)

    return Wzeta #this is an array as well


  w = trapz(Wz, y0, yend, nzeta, Z)
  #print("-----------------------------")
  #print('wy_ff=', w)
  #print("-----------------------------")
  return -(rhot-rhob)*govereightpimu*w


# Stratification Induced Flow Control $\mathscr{w}(x)$

In [ ]:
def wx(yend, X, Y):

  wx_array = np.zeros(len(X))





  wx_array[NearFieldIndicies] = wx_full(X_NF, Y_NF)

  #wx_array[NearFieldIndicies] = wx_FF(X_NF, Y_NF)



  wx_array[FarFieldIndicies] = wx_full(X_FF, Y_FF)

  #wx_array[FarFieldIndicies] = wx_FF(X_FF, Y_FF)


  return wx_array

In [ ]:
def wy(yend, X, Y):



  wy_array = np.zeros(len(X))

  #wy_array[0] = wy_full(X_NF, Y_NF)

  wy_array[NearFieldIndicies] = wy_full(X_NF, Y_NF)

  #print('wy_full far field points',  wy_full(X_FF, Y_FF)[:][0])

  #wy_array[NearFieldIndicies] = wy_FF(X_NF, Y_NF)


  wy_array[FarFieldIndicies] = wy_full(X_FF, Y_FF)

  #wy_array[FarFieldIndicies] = wy_FF(X_FF, Y_FF)







  return wy_array



# Flow Definitions $\mathscr{v}(x)=u(x)+w(x)$

In [ ]:
def vx(yend, X, Y):
  #print('ux=', ux(yend, X, Y))

  vx = ux(yend, X, Y)+wx(yend, X, Y)


  #print('inital vx[0]=', vx[0])


  #vx[0] = abs(ux(yend, X, Y)[0]+wx(yend, X, Y))/2.0

  #vx[0] = ux(yend, X, Y)[0]

  #print('assigned vx[0]', vx[0])

  #vxinterp = np.interp(X, X[1:], vx[1:])

  #vx[0] = vxinterp[0]

  return vx


def vy(yend, X, Y):

  #print('uy=', uy(yend, X, Y))
  #print('inital vy[0]=', vy[0])


  # if yend <-A:

  #   vy = uy(yend, X, Y)




  vy = uy(yend, X, Y)+wy(yend, X, Y)

  #print('vy =', vy)

  #print('vyinterp=', np.interp(X, X[1:], vy[1:]))

  #vyinterp = np.interp(X, X[1:], vy[1:])

  #print('vy=', vy)
  #print('initial vy[0]=', vy[0])
  #print('interpolate vy[0]=', vyinterp[0])

  #vy[0] = vyinterp[0]
  #print(uy(yend, X, Y)[0])


  # vy[0] = uy(yend, X, Y)[0]


  #vy[0] = uy(yend, X, Y)[0]

  #vy[0] = abs(uy(yend, X, Y)+wy(yend, X, Y))

  #vy = uy(yend, X, Y)+wy(yend, X, Y)

  #vy[0] = abs(uy(yend, X, Y)[0]+wy(yend, X, Y))/2.0

  #vy[0] = (vy[2]-vy[1])

  #print('assigned vy[0]=', vy[0])

  return vy




# def vx(yend):

#   return ux(yend)


# def vy(yend):

#   return uy(yend)

# Advection For Loop


In [ ]:
t_array=[]
sphvels = []
yend_array = []
Astar_array = []


wx_array = []
wy_array = []

ux_array = []
uy_array = []

vx_array =[]
vy_array = []




for i in range(Ndt-1):

    yend = Y[i, -1]
    y0 = Y[i,0]
    Astar = abs(Y[i,0])




    r[i, :] = np.sqrt(X[i, :]**2 + Y[i, :]**2)



    # conditions
    conditionNF = r[i,:] < ANF
    # print("-----------------------------")
    #print('ConditionNF=', conditionNF)


    NearFieldIndicies = np.where(conditionNF)[0]
    FarFieldIndicies = np.where(~conditionNF)[0]

    print("-----------------------------")
    print('Time=',i)

    print( 'yend=', yend, 'V_s =', Velocity(yend))




    # print('NF Indicies=',NearFieldIndicies)
    # print('FF Indicies=',FarFieldIndicies)




    print("-----------------------------")


    X_NF = X[i, NearFieldIndicies]
    Y_NF = Y[i, NearFieldIndicies]

    X_FF = X[i, FarFieldIndicies]
    Y_FF = Y[i, FarFieldIndicies]



    #print('vy=', vy(yend, X[i,:], Y[i,:]))

    # X[i+1, 0] = 0.0*X[i,0]
    # Y[i+1, 0] = 0.0*Y[i, 0]

    X[i+1,:] = X[i,:] + dt*vx(yend, X[i, :], Y[i,:])

    Y[i+1, :] = Y[i,:] + dt*vy(yend, X[i, :], Y[i,:])

    # X[i+1,:] = X[i,:] + dt*ux(yend, X[i, :], Y[i,:])

    # Y[i+1, :] = Y[i,:] + dt*uy(yend, X[i, :], Y[i,:])







    t_array.append(i*dt)
    yend_array.append(yend)

    sphvels.append(-1*Velocity(yend))
    Astar_array.append(Astar)

    wx_array.append(wx(yend, X[i,:], Y[i,:]))
    wy_array.append(wy(yend, X[i,:], Y[i,:]))

    ux_array.append(ux(yend, X[i, :], Y[i,:]))
    uy_array.append(uy(yend, X[i, :], Y[i,:]))

    vx_array.append(vx(yend, X[i, :], Y[i,:]))
    vy_array.append(vy(yend, X[i, :], Y[i,:]))

    # if Velocity(yend)>0.05:
    if yend > -2.0*y_start:
    #   break
      break

    # if yend < y0:

    #   break
    #vyinterp_array.append(vyinterp(yend, X[i, :], Y[i,:]))

np.savetxt('{}/dataX.txt'.format(filename), X, delimiter=',', fmt='%s')
np.savetxt('{}/dataY.txt'.format(filename), Y, delimiter=',', fmt='%s')
np.savetxt('{}/AstarArray.txt'.format(filename), Astar_array, delimiter=',', fmt='%s')
np.savetxt('{}/yend_array.txt'.format(filename), yend_array, delimiter=',', fmt='%s')
np.savetxt('{}/sphvels.txt'.format(filename), sphvels, delimiter=',', fmt='%s')
np.savetxt('{}/t_array.txt'.format(filename), t_array, delimiter=',', fmt='%s')
np.savetxt('{}/wx_array.txt'.format(filename), wx_array, delimiter=',',fmt='%f')
np.savetxt('{}/wy_array.txt'.format(filename), wy_array, delimiter=',',fmt='%f')
np.savetxt('{}/ux_array.txt'.format(filename), ux_array, delimiter=',',fmt='%f')
np.savetxt('{}/uy_array.txt'.format(filename), uy_array, delimiter=',',fmt='%f')


  # if Velocity(yend) >0.05:

  #   print('Breaktime=', i)


  #   break

# fieldtitle = 'Far_Field'
# fieldtype = 'Far Field'

fieldtitle = 'FarField_with_Full'
fieldtype = 'Far \,\, Field\, and\, Full'

# fieldtitle = 'FarField_with_no_w'
# fieldtype = 'Far \,\, Field\, and\, No \,W'

# fieldtype = 'Full Field'

# fieldtitle = 'Full_Field'

# fieldtype = 'W=0'

# fieldtitle = 'NoSIF'

infofile = open("{}/InfoFile.txt".format(filename), 'a')
infofile.write('---- 𝐈𝐍𝐅𝐎 𝐅𝐈𝐋𝐄 ----'+'\n'+'\n')
infofile.write('Field Type: {fieldtype}'.format(fieldtype=fieldtype)+'\n'+'\n'+r'Near Field Radius={anf:.2f}'.format(anf=ANF)+'\n'+'\n'+'\n'+r'rho region = {rhoregion}, dt={dt}, T_Max= {Tmax:.2f}, A={A}, rho_s={rhos}, rho_t={rhot}, rho_b={rhob}, mu={mu}'.format(rhoregion=rhoregion, dt=dt, Tmax=Tmax, A=A, rhos=rhos, rhot=rhot, rhob=rhob, mu=mu)+'\n'+'\n'+r'N_i={Ni}, N_j={Nj}, nzeta={nzeta}, nrho={nrho}, ntheta={ntheta}, epsilon={eps}, y_start={ystart}'.format(Ni=Ni, Nj=Nj, nzeta=nzeta, nrho=nrho, ntheta=ntheta, eps = epsilon, ystart=y_start))
infofile.close()







-----------------------------
Time= 0
yend= -1.0 V_s = -0.5629536392259692
-----------------------------
-----------------------------
Time= 1
yend= -0.48816249367848785 V_s = -0.4752069396014018
-----------------------------
-----------------------------
Time= 2
yend= -0.04766580311165958 V_s = -0.39567590515729967
-----------------------------
-----------------------------
Time= 3
yend= 0.3174725070858247 V_s = -0.3298290845846267
-----------------------------
-----------------------------
Time= 4
yend= 0.6188712533678641 V_s = -0.2977992481165122
-----------------------------
-----------------------------
Time= 5
yend= 0.8901577223282364 V_s = -0.3002214961036533
-----------------------------
-----------------------------
Time= 6
yend= 1.1632452819304138 V_s = -0.3002414361987894
-----------------------------
-----------------------------
Time= 7
yend= 1.4361199629962753 V_s = -0.30024192703924507
-----------------------------


# Interface Plotting

In [ ]:
k= 500

In [ ]:
fig, ax = plt.subplots( figsize=(10,10), sharex=True, sharey=False)



ax.set_xlabel("x-position")
ax.set_ylabel("y-position")

# Plotting a sphere of radius

sphx = np.linspace(0,A, 100)
sphyp = np.sqrt(A**2- sphx**2)
sphym = - sphyp
ax.set(adjustable='box', aspect='equal')

ax.plot(sphx, sphyp, 'k', sphx, sphym, 'k', -sphx, -sphyp, 'k', -sphx, sphyp, 'k', lw=2)



# plt.vlines(x=A, ymin=y_start*1.2, ymax=-y_start*1.5, linestyle='dashed', lw=0.5, color='blue', alpha=0.5, label=r'$\pm A$')
# plt.vlines(x=-A, ymin=y_start*1.2, ymax=-y_start*1.5, linestyle='dashed', lw=0.5, color='blue', alpha=0.5)


# for i in range(len(t_array)-1):
for i in range( Ndt-1):
  if i % k ==0:
    plt.plot(X[i, :],Y[i,:], c='tab:blue', lw=0.8)
    plt.plot(-X[i, :],Y[i,:], c='tab:blue', lw=0.8)

# plt.scatter(X[-1, 1:],Y[-1,1:], c='tab:green', s=8.0)
# plt.scatter(-X[-1, :],Y[-1,:], c='tab:green', s=8.0)
plt.plot(X[-1, :],Y[-1,:], c='tab:red', lw=0.8)
plt.plot(-X[-1, :],Y[-1,:], c='tab:red', lw=0.8)

sphxn = np.linspace(0,ANF, 100)
sphypn = np.sqrt(ANF**2- sphxn**2)
sphymn = - sphypn
ax.set(adjustable='box', aspect='equal')
ax.plot(sphxn, sphypn, 'tab:orange', linestyle='dashed', lw=1.0,  label='ANF={anf}'.format(anf=ANF))
ax.plot(sphxn, sphymn, 'tab:orange', -sphxn, -sphypn, 'tab:orange', -sphxn, sphypn, 'tab:orange', linestyle='dashed', lw=1.0)




ax.set_xlim([-R0*1.01,R0*1.01])
ax.set_ylim([Y[0,0]*1.2, -Y[0,0]*1.5])
# ax.set(adjustable='box', aspect='equal')

plt.savefig("{}/EulerStokes_FlowPlot.png".format(filename),bbox_inches ="tight", facecolor='white')
plt.legend()






In [ ]:
#X[1]

# Metric Plotting

In [ ]:
l=30


**markdown**


In [ ]:


plt1, axes = plt.subplots()
V_TermTop =  (2*A**2 *g)/(9*mu)*(rhot-rhos)
V_TermBttm = (2*A**2 *g)/(9*mu)*(rhob-rhos)


plt.title('Sphere Velocity vs. Time')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (cm/s)')

plt.plot(t_array, sphvels, color='tab:blue', label='Simulation')

plt.axhline(y= -V_TermBttm, color='tab:orange', linestyle='dashed', alpha=0.9, label = r'$V_{Term,Bottom}$')

plt.axhline(y= -V_TermTop, color='tab:grey', linestyle=':', alpha=0.5, label = r'$V_{Term,Top}$')
#plt.axhline(y= 0, color='tab:grey', linestyle=':', alpha=0.2)

plt.legend()

plt.savefig("{filename}/Velocity_v_time.png".format(filename=filename),bbox_inches ="tight", facecolor='white')


plt2, axes = plt.subplots()
V_TermTop =  (2*A**2 *g)/(9*mu)*(rhot-rhos)
V_TermBttm = (2*A**2 *g)/(9*mu)*(rhob-rhos)


plt.title('Sphere Velocity vs. yend')
plt.xlabel('yend (cm)')
plt.ylabel('Velocity (cm/s)')

plt.plot(yend_array, sphvels, color='tab:blue', label='Simulation')

plt.axhline(y= -V_TermBttm, color='tab:orange', linestyle='dashed', alpha=0.9, label = r'$V_{Term,Bottom}$')

plt.axhline(y= -V_TermTop, color='tab:grey', linestyle=':', alpha=0.5, label = r'$V_{Term,Top}$')

plt.axvline(x=A, color = 'tab:green', linestyle='dashed', label=r'$\pm A$')
plt.axvline(x=-A, color='tab:green', linestyle='dashed')

plt.axvline(x=ANF, color = 'tab:pink', linestyle='dashed', label=r'$\pm ANF$')
plt.axvline(x=-ANF, color='tab:pink', linestyle='dashed')



plt.legend()

plt.savefig("{filename}/Velocity_v_yend.png".format(filename=filename),bbox_inches ="tight", facecolor='white')


plt3, axes = plt.subplots()

K=0

# Plot a single point of Y over time
plt.title(r'$y_0$ vs. time')
plt.xlabel('time')
plt.ylabel('Y')

#plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

plt.plot(np.linspace(0, Tmax, len(Y[:,0])), Y[:, 0], color='k')

#plt.xticks([])

plt.legend()

plt.savefig("{filename}/y0_v_timne.png".format(filename=filename),bbox_inches ="tight", facecolor='white')




plt3, axes = plt.subplots()

plt.title(r'A* vs. time')
plt.xlabel('time')
plt.ylabel('A*')

# plt.hlines(y=A, xmin=-100, xmax = Tmax, linestyle='dashed', color='tab:orange', label='A')
# plt.hlines(y=ANF, xmin=0, xmax = Tmax, linestyle='dashed', color='tab:pink', label='ANF')

plt.axhline(y= A, linestyle='dashed', color='tab:orange', label='A')
plt.axhline(y= ANF, linestyle='dashed', color='tab:pink', label='ANF')


plt.plot(np.linspace(0, Tmax, len(Astar_array)), Astar_array)

plt.legend()

plt.savefig("{filename}/Astar_v_time.png".format(filename=filename),bbox_inches ="tight", facecolor='white')

plt4, axes = plt.subplots()

plt.title(r'A* vs. yend')
plt.xlabel('yend')
plt.ylabel('A*')





plt.axvline(x=A, color = 'tab:green', linestyle='dashed', label=r'$\pm A$')
plt.axvline(x=-A, color='tab:green', linestyle='dashed')
plt.axhline(y=A, color = 'tab:green', linestyle='dashed')
plt.axhline(y=-A, color='tab:green', linestyle='dashed')



plt.axvline(x=ANF, color = 'tab:pink', linestyle='dashed', label=r'$\pm A_{NF}$')
plt.axvline(x=-ANF, color='tab:pink', linestyle='dashed')
plt.axhline(y=ANF, color = 'tab:pink', linestyle='dashed')
plt.axhline(y=-ANF, color='tab:pink', linestyle='dashed')

plt.plot(yend_array, Astar_array)

axes.set_xlim([1.02*y_start, -2.0*y_start])

plt.legend()

plt.savefig("{filename}/Astar_v_yend.png".format(filename = filename),bbox_inches ="tight", facecolor='white')



plt5a, axes = plt.subplots()

plt.title(r'$wx\,\, vs.\,\, X$')
plt.xlabel('X')
plt.ylabel(r'$wx$')


plt.axvline(x=A, color = 'tab:grey', linestyle='dashed', label=r'$\pm A$')


for eta in range(Tmax-1):

  if eta % l == 0:

    plt.plot(X[eta, :], wx_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()
plt.savefig("{}/wx_v_interface.png".format(filename),bbox_inches ="tight", facecolor='white')
plt5, axes = plt.subplots()

plt.title(r'$wy \,\, vs.\,\, X$')
plt.xlabel('X')
plt.ylabel(r'$wy$')

plt.axvline(x=A, color = 'tab:grey', linestyle='dashed', label=r'$\pm A$')



for eta in range(Tmax-1):

  if eta % l ==0:

    plt.plot(X[eta, :], wy_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()

plt.savefig("{}/wy_v_interface.png".format(filename),bbox_inches ="tight", facecolor='white')
# plt6, axes = plt.subplots()

# plt.title(r'$w \,\, vs.\,\, time$')
# plt.xlabel('time')
# plt.ylabel(r'$w$')


# for eta in range(Tmax-1):

#   if eta % l == 0:

#     plt.plot(np.linspace(0, Ndt, len(wx_array[eta]+wy_array[eta])), wx_array[eta]+wy_array[eta], label=r'$time={eta}$'.format(eta=eta))


# plt.legend()


plt7, axes= plt.subplots()
plt.title(r'$ux\,\, vs.\,\, X$')
plt.xlabel('X')
plt.ylabel(r'$ux$')

plt.axvline(x=A, color = 'tab:green', linestyle='dashed', label=r'$\pm A$')


for eta in range(Tmax-1):

  if eta % l == 0:

    plt.plot(X[eta,:], ux_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()
plt.savefig("{}/ux_v_interface.png".format(filename),bbox_inches ="tight", facecolor='white')

plt8, axes = plt.subplots()

plt.title(r'$uy \,\, vs.\,\, X$')
plt.xlabel('X')
plt.ylabel(r'$uy$')



for eta in range(Tmax-1):

  if eta % l ==0:

    plt.plot(X[eta,:], uy_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()
plt.savefig("{}/uy_v_interface.png".format(filename),bbox_inches ="tight", facecolor='white')
plt9, axes = plt.subplots()

plt.title(r'$u \,\, vs.\,\, X$')
plt.xlabel('X')
plt.ylabel(r'$u$')

for eta in range(Tmax-1):

  if eta % l == 0:
    #plt.scatter(X[eta,:], ux_array[eta]+uy_array[eta])
    plt.plot(X[eta], ux_array[eta]+uy_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()


plt10, axes = plt.subplots()

plt.title(r'$w \,\, vs.\,\, X$')
plt.xlabel('X')
plt.ylabel(r'$w$')
plt.axvline(x=A, color = 'tab:green', linestyle='dashed', label=r'$\pm A$')
#plt.vlines(x=-A, ymin=0, ymax=1e-4, linestyle='dashed', color='grey', alpha=0.5, label='-A')


#eta = 2

for eta in range(Tmax-1):

  if eta % l == 0:
    #plt.scatter(X[eta,:], wx_array[eta]+wy_array[eta])
    plt.plot(X[eta], wx_array[eta]+wy_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()


plt10, axes = plt.subplots()

plt.title(r'$vx \,\, over \,\, entrainment$')
plt.xlabel('X')
plt.ylabel(r'$vx$')

#plt.vlines(x=-A, ymin=0, ymax=max(S_int_array), linestyle='dashed', color='grey', alpha=0.5, label='-A')


#eta = 2

for eta in range(Tmax-1):

  if eta % l == 0:

    plt.plot(X[eta, :], vx_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()
plt.savefig("{}/vx_v_interface.png".format(filename),bbox_inches ="tight", facecolor='white')

plt11, axes = plt.subplots()

plt.title(r'$vy \,\, over \,\, entrainment$')
plt.xlabel('X')
plt.ylabel(r'$vy$')

#plt.vlines(x=-A, ymin=0, ymax=max(S_int_array), linestyle='dashed', color='grey', alpha=0.5, label='-A')


#eta = 2

for eta in range(Tmax-1):

  if eta % l == 0:

    plt.plot(X[eta, :], vy_array[eta], label=r'$time={eta}$'.format(eta=eta))


plt.legend()

plt.savefig("{}/vy_v_interface.png".format(filename),bbox_inches ="tight", facecolor='white')

plt12, axes = plt.subplots()

plt.title(r'$v \,\, over \,\, entrainment$')
plt.xlabel('X')
plt.ylabel(r'$v$')

plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)

# plt.vlines(x=A+epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9, label=r'$A \pm \epsilon$')

# plt.vlines(x=A-epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9)

#eta = 2

for eta in range(Tmax-1):

  if eta % l == 0:
    #plt.scatter(X[eta,:], vx_array[eta]+vy_array[eta])

    # plt.vlines(x=X[eta, 3], ymin=min(vx_array[eta]+vy_array[eta]), ymax=max(vx_array[eta]+vy_array[eta]))
    plt.plot(X[eta, :], abs(vx_array[eta]**2+vy_array[eta]**2)/2.0, label=r'$time={eta}$'.format(eta=eta))


plt.legend()


# ksi=0

# plt13, axes = plt.subplots()

# plt.title(r'$wx \,\, at\,\, X[{ksi}] \,\, over \,\, time$'.format(ksi=ksi))
# plt.xlabel('time'.format(ksi=ksi))
# plt.ylabel(r'$wx$')

# #plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)


# #plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

# for i in np.arange(0, Tmax, 1):
#   plt.scatter(i, wx_array[i][ksi], color='k', s=3.9)



# plt.legend()
# plt.show()



# plt14, axes = plt.subplots()

# plt.title(r'$wy \,\, at\,\, X[{ksi}] \,\, over \,\, time$'.format(ksi=ksi))
# plt.xlabel('time'.format(ksi=ksi))
# plt.ylabel(r'$wy$')

# #plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)


# #plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

# for i in np.arange(0, Tmax, 1):
#   plt.scatter(i, wy_array[i][ksi], color='k', s=3.9)



# plt.legend()
# plt.show()


# plt13, axes = plt.subplots()

# plt.title(r'$w \,\, at\,\, X[{ksi}] \,\, over \,\, time$'.format(ksi=ksi))
# plt.xlabel('time'.format(ksi=ksi))
# plt.ylabel(r'$w$')

# #plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)


# #plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

# for i in np.arange(0, Tmax, 1):
#   plt.scatter(i, wx_array[i][ksi]+wy_array[i][ksi], color='k', s=3.9)


8
# plt.legend()
# plt.show()




In [ ]:
# ksi=0

# plt13, axes = plt.subplots()

# plt.title(r'$wx \,\, at\,\, X[{ksi}] \,\, over \,\, time$'.format(ksi=ksi))
# plt.xlabel('time'.format(ksi=ksi))
# plt.ylabel(r'$wx$')

# #plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)


# #plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

# for i in np.arange(0, Tmax, 1):
#   plt.scatter(i, wx_array[i][ksi], color='k', s=3.9)



# plt.legend()
# plt.show()

In [ ]:
# ksi=0

# plt13, axes = plt.subplots()

# plt.title(r'$wy \,\, at\,\, X[{ksi}] \,\, over \,\, time$'.format(ksi=ksi))
# plt.xlabel('time'.format(ksi=ksi))
# plt.ylabel(r'$wy$')

# #plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)


# #plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

# for i in np.arange(0, Tmax, 1):
#   plt.scatter(i, wy_array[i][ksi]+wy_array[i][ksi], color='k', s=3.9)



# plt.legend()
# plt.show()

In [ ]:
# ksi=0

# plt14, axes = plt.subplots()

# plt.title(r'$wy \,\, at\,\, X[{ksi}] \,\, over \,\, time$'.format(ksi=ksi))
# plt.xlabel('time'.format(ksi=ksi))
# plt.ylabel(r'$wy$')

# #plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)


# #plt.scatter(np.linspace(0, Tmax, len(Y[:,K])), Y[:, K], c=Y[:,K], cmap='gist_gray' )

# for i in np.arange(0, Tmax, 1):
#   plt.scatter(i, wy_array[i][ksi], color='k', s=3.9)



# plt.legend()
# plt.show()

In [ ]:
Ndt

In [ ]:
plt12f, axes = plt.subplots()

plt.title(r'$velocities \,\, x \,\, over \,\, entrainment$')
plt.xlabel('X')
plt.ylabel(r'$horizontal \, velocities $')

plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)

# plt.vlines(x=A+epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9, label=r'$A \pm \epsilon$')

# plt.vlines(x=A-epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9)

eta = int(len(t_array)/2.0)

plt.plot(X[eta, :], ux_array[eta], label=r'$u$')
plt.plot(X[eta, :], wx_array[eta], label=r'$w$')
plt.plot(X[eta, :], vx_array[eta], label=r'$v$')


plt.legend()

In [ ]:
plt12d, axes = plt.subplots()

plt.title(r'$velocities \,\, y \,\, over \,\, entrainment$')
plt.xlabel('X')
plt.ylabel(r'$vertical \, velocities $')

plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)

# plt.vlines(x=A+epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9, label=r'$A \pm \epsilon$')

# plt.vlines(x=A-epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9)

eta = int(len(t_array)*0.5)

plt.plot(X[eta, :], uy_array[eta], label=r'$u$')
plt.plot(X[eta, :], wy_array[eta], label=r'$w$')
plt.plot(X[eta, :], vy_array[eta], label=r'$v$')


plt.legend()

In [ ]:
len(t_array)

In [ ]:
plt12f, axes = plt.subplots()

plt.title(r'$velocities \,\, \,\, over \,\, entrainment$')
plt.xlabel('X')
plt.ylabel(r'$ Velocities $')

plt.hlines(y=0, xmin=-0.01, xmax=R0, linestyle='dashed', color='grey', alpha=0.9)

# plt.vlines(x=A+epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9, label=r'$A \pm \epsilon$')

# plt.vlines(x=A-epsilon, ymin=0, ymax=0.26, linestyle='dashed', color='tab:orange', alpha=0.9)

eta = int(len(t_array)/2.0)

plt.plot(X[eta, :], np.sqrt(ux_array[eta]**2 + uy_array[eta]**2), label=r'$|u|$')
plt.plot(X[eta, :], np.sqrt(wx_array[eta]**2+wy_array[eta]**2), label=r'$|w|$')
plt.plot(X[eta, :], np.sqrt(vx_array[eta]**2+vy_array[eta]**2), label=r'$|v|$')


plt.legend()

In [ ]:
print(Ndt)

In [ ]:
# eval_time = Ndt-9

eval_time =int(len(t_array)*0.55)

In [ ]:
X_to_plot = X[eval_time, :]
Y_to_plot = Y[eval_time, :]

In [ ]:
  # Plotting a sphere of radius

fig, ax = plt.subplots( figsize=(10,10), sharex=True, sharey=False)
plt.title('Single Entrainment'+'\n'+r'Time={t}'.format(t=eval_time))

sphx = np.linspace(0,A, 100)
sphyp = np.sqrt(A**2- sphx**2)
sphym = - sphyp


ax.plot(sphx, sphyp, 'k',lw=2, label='Sphere')
ax.plot(sphx, sphym, 'k', -sphx, -sphyp, 'k', -sphx, sphyp, 'k', lw=2)

sphxs = np.linspace(0,Astar, 100)
sphyps = np.sqrt(Astar**2- sphxs**2)
sphyms = - sphyps

ax.plot(sphxs, sphyps, 'tab:orange', alpha=0.5, lw=2, ls='--')

ax.plot(sphxs, sphyms, 'tab:orange', alpha=0.5, lw=2,ls='--')
ax.plot(-sphxs, -sphyps, 'tab:orange', alpha=0.5, lw=2, ls='--')
ax.plot(-sphxs, sphyps, 'tab:orange', alpha=0.5, lw=2,ls='--', label='Shell')

# plot ANF

sphxn = np.linspace(0,ANF, 100)
sphyn = np.sqrt(ANF**2- sphxn**2)
sphymn = - sphyn


ax.plot(sphxn, sphyn, 'tab:green', alpha=0.5, lw=1.5, ls='-.')

ax.plot(sphxn, sphymn, 'tab:green', alpha=0.5, lw=1.5,ls='-.')
ax.plot(-sphxn, -sphyn, 'tab:green', alpha=0.5, lw=1.5, ls='-.')
ax.plot(-sphxn, sphyn, 'tab:green', alpha=0.5, lw=1.5,ls='-.', label='ANF')




# plt.hlines(y=-A, xmin=-R0, xmax=R0, color='tab:brown', ls='dotted', label=r'$y=-A$')

ax.plot(X_to_plot, Y_to_plot, color='tab:blue', label='Entrainment: Simulation')
ax.plot(-X_to_plot, Y_to_plot, color='tab:blue')

plt.legend()

# ax.set_xlim([-1,1])
# ax.set_ylim([-3, -1])


ax.set(adjustable='box', aspect='equal')
fig.tight_layout()




In [ ]:
len(t_array)

In [ ]:

# eta = 1000

# #eta = int((Ndt-2)/2.0)

# eta2 = 50

# eta3 = Ndt-3

# fig, axes = plt.subplots()

# plt.suptitle(r'$Velocity \,\, Comparisons \,\, vs.\,\, X$')

# #plt.title(r'$ At \,\, t_1={eta} ,\,\, t_2={eta2} \,\, and \,\, t_3={eta3}$'.format(eta=eta, eta2=eta2, eta3=eta3))

# plt.title(r'$ At \,\, t_1={eta}$'.format(eta=eta))

# plt.xlabel('X')
# plt.ylabel(r'$Velocities_y$')

# #plt.vlines(x=-A, ymin=0, ymax=max(S_int_array), linestyle='dashed', color='grey', alpha=0.5, label='-A')



# # plt.plot(X[eta,:], uy_array[eta], label=r'$uy(t_1)$', color='k')
# # plt.plot(X[eta,:], wy_array[eta], ls='dashed', label=r'$wy(t_1)$', color='grey')
# # plt.plot(X[eta,:], vy_array[eta], ls='-.', label=r'$vy(t_1)$', color='lightgrey')

# # # plt.plot(X[eta2,:], uy_array[eta2],alpha=1.0, label=r'$uy(t_2)$', color='k')
# # # plt.plot(X[eta2,:], wy_array[eta2],alpha=1.0, ls='dashed', label=r'$wy(t_2)$', color='grey')
# # # plt.plot(X[eta2,:], vy_array[eta2],alpha=1.0, ls='-.', label=r'$vy(t_2)$', color='lightgrey')

# # plt.plot(X[eta3,:], uy_array[eta3],alpha=0.9, label=r'$uy(t_3)$', color='k')
# # plt.plot(X[eta3,:], wy_array[eta3],alpha=0.9, ls='dashed', label=r'$wy(t_3)$', color='grey')
# # plt.plot(X[eta3,:], vy_array[eta3],alpha=0.9, ls='-.', label=r'$vy(t_3)$', color='lightgrey')




# # #plt.plot(X[eta,:], vy_array[eta], label=r'$vy(t_1)$', color='tab:brown')


# plt.plot(X[eta,:], uy_array[eta], alpha=0.9,  label=r'$uy(t_1)$', color='tab:blue')
# plt.plot(X[eta,:], wy_array[eta], alpha=0.9, ls='dashed', label=r'$wy(t_1)$', color='tab:orange')
# plt.plot(X[eta,:], vy_array[eta], ls='dotted', label=r'$vy(t_1)$', color='indianred')


# # plt.plot(X[eta2,:], uy_array[eta2], alpha=0.9, ls='dashed', label=r'$uy(t_2)$', color='#114266')
# # plt.plot(X[eta2,:], wy_array[eta2], alpha=0.9, ls='dashed', label=r'$wy(t_2)$', color='#9A4C08')
# #plt.plot(X[eta2,:], vy_array[eta2], label=r'$vy(t_2)$', color='indianred')


# #plt.plot(X[eta2,:], vy_array[eta2], alpha=0.9, ls='dashed', label=r'$vy(t_2)$', color='tab:brown')

# # plt.plot(X[eta3,:], uy_array[eta3], alpha=0.9, ls='-.', label=r'$uy(t_3)$', color='#092133')
# # plt.plot(X[eta3,:], wy_array[eta3], alpha=0.9, ls='-.', label=r'$wy(t_3)$', color='#663305')

# # plt.plot(X[eta3,:], vy_array[eta3], alpha=0.9, ls='-.', label=r'$vy(t_3)$', color='darkred')



# plt.legend(loc='upper right')